In [ ]:
pip install --upgrade pip

In [ ]:
pip install tensorflow

In [ ]:
pip install pyvi

In [ ]:
pip install gensim

In [9]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [15]:
import zipfile

zip_ref = zipfile.ZipFile("C:/Users/bangp/Downloads/VNTC-master.zip", 'r')
zip_ref.extractall("/content/DATA")
zip_ref.close()


In [10]:
pip install patool

  Using cached patool-1.12-py2.py3-none-any.whl (77 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:

import patoolib

In [21]:
patoolib.extract_archive("/content/DATA/VNTC-master/Data/10Topics/Ver1.1/Train_Full.rar", outdir="/content/DATA")

patool: Extracting /content/DATA/VNTC-master/Data/10Topics/Ver1.1/Train_Full.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- c:\content\DATA\VNTC-master\Data\10Topics\Ver1.1\Train_Full.rar
patool:     with cwd=/content/DATA
patool: ... /content/DATA/VNTC-master/Data/10Topics/Ver1.1/Train_Full.rar extracted to `/content/DATA'.


'/content/DATA'

In [12]:
import os 
# dir_path = os.getcwd()

In [25]:


def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)

                X.append(lines)
                y.append(path)

    return X, y

train_path = os.path.join('c:\content\DATA', 'Train_Full')
X_data, y_data = get_data(train_path)

100%|██████████| 10/10 [06:18<00:00, 37.86s/it]


In [30]:
import pickle

pickle.dump(X_data, open('c:\content\DATA\X_data.pkl', 'wb'))
pickle.dump(y_data, open('c:\content\DATA/y_data.pkl', 'wb'))

In [32]:
patoolib.extract_archive("/content/DATA/VNTC-master/Data/10Topics/Ver1.1/Test_Full.rar", outdir="/content/DATA")

patool: Extracting /content/DATA/VNTC-master/Data/10Topics/Ver1.1/Test_Full.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- c:\content\DATA\VNTC-master\Data\10Topics\Ver1.1\Test_Full.rar
patool:     with cwd=/content/DATA
patool: ... /content/DATA/VNTC-master/Data/10Topics/Ver1.1/Test_Full.rar extracted to `/content/DATA'.


'/content/DATA'

In [34]:
test_path = os.path.join('c:\content\DATA', 'Test_Full')
X_test, y_test = get_data(test_path)

pickle.dump(X_test, open('c:\content\DATA/X_test.pkl', 'wb'))
pickle.dump(y_test, open('c:\content\DATA/y_test.pkl', 'wb'))

100%|██████████| 10/10 [19:04<00:00, 114.50s/it]


In [13]:
import pickle

X_data = pickle.load(open('c:\content\DATA/X_data.pkl', 'rb'))
y_data = pickle.load(open('c:\content\DATA/y_data.pkl', 'rb'))

X_test = pickle.load(open('c:\content\DATA/X_test.pkl', 'rb'))
y_test = pickle.load(open('c:\content\DATA/y_test.pkl', 'rb'))


In [14]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)


In [16]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


In [38]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)


In [39]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)


In [17]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)


In [41]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)


In [42]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)


In [18]:

from gensim.models import KeyedVectors 
# dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join('c:\content\DATA', "vi/vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.key_to_index
wv = w2v

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                sentence.append(wv[word])

        word2vec_data.append(sentence)

    return word2vec_data

X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)


In [19]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import  metrics
from sklearn import naive_bayes

In [20]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [11]:
encoder.classes_

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

In [28]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
    return classifier
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))


In [23]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả
# Train accuracy:  0.880031596616529
# Validation accuracy:  0.8690758293838863
# Test accuracy:  0.8650666031405714


Validation accuracy:  0.8524881516587678
Test accuracy:  0.8628034859944812


In [24]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras import layers, models, optimizers
from keras.layers import *

In [25]:
def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [29]:
classifier = create_dnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)

Epoch 1/3
60/60 [==============================] - 4s 58ms/step - loss: 0.7637 - accuracy: 0.7735 - val_loss: 0.3354 - val_accuracy: 0.8827
Epoch 2/3
60/60 [==============================] - 3s 51ms/step - loss: 0.2604 - accuracy: 0.9131 - val_loss: 0.3005 - val_accuracy: 0.8969
Epoch 3/3
1575/1575 [==============================] - 6s 4ms/step
Validation accuracy:  0.9066943127962085
Test accuracy:  0.9095944255851349


In [33]:
def create_brnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier


In [40]:
classifier = create_brnn_model()
model = train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=20)

Epoch 1/20
60/60 [==============================] - 39s 637ms/step - loss: 1.4940 - accuracy: 0.5315 - val_loss: 0.8415 - val_accuracy: 0.7367
Epoch 2/20
60/60 [==============================] - 9s 141ms/step - loss: 0.5644 - accuracy: 0.8199 - val_loss: 0.5004 - val_accuracy: 0.8315
Epoch 3/20
60/60 [==============================] - 8s 126ms/step - loss: 0.4385 - accuracy: 0.8551 - val_loss: 0.4531 - val_accuracy: 0.8469
Epoch 4/20
60/60 [==============================] - 8s 130ms/step - loss: 0.4023 - accuracy: 0.8653 - val_loss: 0.4440 - val_accuracy: 0.8501
Epoch 5/20
60/60 [==============================] - 8s 127ms/step - loss: 0.3799 - accuracy: 0.8729 - val_loss: 0.4002 - val_accuracy: 0.8670
Epoch 6/20
60/60 [==============================] - 8s 141ms/step - loss: 0.3501 - accuracy: 0.8822 - val_loss: 0.3846 - val_accuracy: 0.8679
Epoch 7/20
60/60 [==============================] - 9s 154ms/step - loss: 0.3430 - accuracy: 0.8847 - val_loss: 0.3772 - val_accuracy: 0.8682
Epoch

In [42]:
classifier.save('models/model_brnn')

None
INFO:tensorflow:Assets written to: models/model_brnn\assets
None


In [35]:
def create_rcnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu', return_sequences=True))(layer)    
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [36]:
classifier = create_rcnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=20)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 300)]             0         
                                                                 
 reshape_2 (Reshape)         (None, 10, 30)            0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 256)          122880    
 nal)                                                            
                                                                 
 conv1d (Conv1D)             (None, 8, 100)            76900     
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense_24 (Dense)            (None, 512)               410112    
                                                           